# Readme:

This notebook is a demo of GCN (Global Conv Net) for the CSC413 Final Project created by Sanzhe Feng.

Original Paper: https://arxiv.org/abs/1703.02719



# Goal

As mentioned in the report, we cannot fully constructed a GCN due to time and technical restrictions including things like data access. Here we performed a simple demo to just show the doablility of GCN and give the readers a general idea of how can we apply GCN.

However, since we are using CIFAR10 dataset, GCN is not a good choice to do classification and thus will not give satisfying results.

Again, the idea of the notebook is just to show what does a GCN model look like.



In [1]:

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import CIFAR10

# Define the Global Convolution Network (GCN) architecture
class GlobalConvolutionNetwork(nn.Module):
    """
    Global Convolution Network which can be regarded as Spatial-wise attention.
    """
    def __init__(self, in_channels, out_channels, k=3, padding=1):
        super(GlobalConvolutionNetwork, self).__init__()

        self.conv_l1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(k, 1), padding=(padding, 0))
        self.conv_l2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=(1, k), padding=(0, padding))

        self.conv_r1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1, k), padding=(0, padding))
        self.conv_r2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=(k, 1), padding=(padding, 0))

        # Add an adaptive average pooling layer to reduce spatial dimensions to 1x1
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x1 = self.conv_l1(x)
        x1 = self.conv_l2(x1)

        x2 = self.conv_r1(x)
        x2 = self.conv_r2(x2)

        out = x1 + x2

        # Apply the adaptive average pooling and flatten the tensor
        out = self.adaptive_avg_pool(out)
        out = out.view(out.size(0), -1)

        return out


# Define a custom dataset and data loader
class CustomCIFAR10(Dataset):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        self.cifar10 = CIFAR10(root, train, transform, target_transform, download)

    def __getitem__(self, index):
        return self.cifar10[index]

    def __len__(self):
        return len(self.cifar10)

# Define training and evaluation functions
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return running_loss / len(dataloader), correct / total

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return running_loss / len(dataloader), correct / total


KeyboardInterrupt: ignored

In [ ]:
def main():
    # Set hyperparameters
    num_epochs = 100
    batch_size = 100
    learning_rate = 0.001

    # Prepare data
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = CustomCIFAR10(root='./data', train=True, transform=transform, download=True)
    test_dataset = CustomCIFAR10(root='./data', train=False, transform=transform, download=True)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Set up the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize the model, criterion, and optimizer
    model = GlobalConvolutionNetwork(3, 10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train and evaluate the model
    for epoch in range(num_epochs):
        train_loss, train_acc = train(model, train_dataloader, criterion, optimizer, device)
        test_loss, test_acc = evaluate(model, test_dataloader, criterion, device)

        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

    # Save the trained model
    torch.save(model.state_dict(), "gcn_model.pth")
    print("Model saved successfully.")

if __name__ == "__main__":
    main()


100%|██████████| 170498071/170498071 [00:02<00:00, 61655844.76it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/100], Train Loss: 2.1612, Train Acc: 0.1943, Test Loss: 2.0818, Test Acc: 0.2272
Epoch [2/100], Train Loss: 2.0798, Train Acc: 0.2222, Test Loss: 2.0621, Test Acc: 0.2321
Epoch [3/100], Train Loss: 2.0719, Train Acc: 0.2225, Test Loss: 2.0589, Test Acc: 0.2340
Epoch [4/100], Train Loss: 2.0707, Train Acc: 0.2232, Test Loss: 2.0593, Test Acc: 0.2312
Epoch [5/100], Train Loss: 2.0700, Train Acc: 0.2245, Test Loss: 2.0565, Test Acc: 0.2336
Epoch [6/100], Train Loss: 2.0700, Train Acc: 0.2249, Test Loss: 2.0551, Test Acc: 0.2303
Epoch [7/100], Train Loss: 2.0689, Train Acc: 0.2231, Test Loss: 2.0543, Test Acc: 0.2367
Epoch [8/100], Train Loss: 2.0692, Train Acc: 0.2256, Test Loss: 2.0609, Test Acc: 0.2340
Epoch [9/100], Train Loss: 2.0684, Train Acc: 0.2265, Test Loss: 2.0567, Test Acc: 0.2327
Epoch [10/100], Train Loss: 2.0685, Train Acc: 0.2242, Test Loss: 2.0560, Test Acc: 0.2294
Epoch [11/